# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Leafnumber

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListArray(a, e)
        TYPE(container),ALLOCATABLE,INTENT(INOUT) :: a(:)
        CLASS(*),INTENT(IN), allocatable :: e(:)
        TYPE(container),ALLOCATABLE :: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(1)%items(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+1))
            a(1:SIZE(tmp)) = tmp
            ALLOCATE(a(SIZE(tmp)+1)%items(SIZE(e)), source = e)
        END IF
    END SUBROUTINE AddToListArray

END MODULE list_sub
MODULE Leafnumbermod
    IMPLICIT NONE
CONTAINS
    SUBROUTINE model_leafnumber(deltaTT, &
        phyllochron, &
        hasFlagLeafLiguleAppeared, &
        leafNumber, &
        phase)
        REAL, INTENT(IN) :: deltaTT
        REAL, INTENT(IN) :: phyllochron
        INTEGER, INTENT(IN) :: hasFlagLeafLiguleAppeared
        REAL, INTENT(INOUT) :: leafNumber
        REAL, INTENT(IN) :: phase
        REAL:: phyllochron_
        !- Description:
    !            - Model Name: CalculateLeafNumber Model
    !            - Author: Pierre MARTRE
    !            - Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            - Institution: INRA Montpellier
    !            - Abstract: calculate leaf number. LeafNumber increase is caped at one more leaf per day
        !- inputs:
    !            - name: deltaTT
    !                          - description : daily delta TT 
    !                          - variablecategory : auxiliary
    !                          - datatype : DOUBLE
    !                          - min : -20
    !                          - max : 100
    !                          - default : 23.5895677277199
    !                          - unit : °C d
    !                          - inputtype : variable
    !            - name: phyllochron
    !                          - description : phyllochron
    !                          - variablecategory : state
    !                          - inputtype : variable
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 1000
    !                          - default : 0
    !                          - unit : °C d leaf-1
    !            - name: hasFlagLeafLiguleAppeared
    !                          - description : true if flag leaf has appeared (leafnumber reached finalLeafNumber)
    !                          - variablecategory : state
    !                          - datatype : INT
    !                          - min : 0
    !                          - max : 1
    !                          - default : 0
    !                          - unit : 
    !                          - inputtype : variable
    !            - name: leafNumber
    !                          - description :  Actual number of phytomers
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 25
    !                          - default : 0
    !                          - unit : leaf
    !                          - inputtype : variable
    !            - name: phase
    !                          - description :  the name of the phase
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 7
    !                          - default : 1
    !                          - unit :  
    !                          - uri : some url
    !                          - inputtype : variable
        !- outputs:
    !            - name: leafNumber
    !                          - description : Actual number of phytomers
    !                          - variablecategory : state
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 10000
    !                          - unit : leaf
    !                          - uri : some url
        IF(phase .GE. 1.0 .AND. phase .LT. 4.0) THEN
            IF(hasFlagLeafLiguleAppeared .EQ. 0) THEN
                IF(phyllochron .EQ. 0.0) THEN
                    phyllochron_ = 0.0000001
                ELSE
                    phyllochron_ = phyllochron
                END IF
                leafNumber = leafNumber + MIN(deltaTT / phyllochron_, 0.999)
            END IF
        END IF
    END SUBROUTINE model_leafnumber

END MODULE
PROGRAM test
    USE Leafnumbermod
    REAL:: deltaTT
    REAL:: phyllochron
    INTEGER:: hasFlagLeafLiguleAppeared
    REAL:: leafNumber
    REAL:: phase
    print *, "--------test_test_wheat1_LeafNumber-------"
    leafNumber = 5.147163833893262
    phase = 3
    phyllochron = 91.2
    deltaTT = 23.5895677277199
    hasFlagLeafLiguleAppeared = 0
    call model_leafnumber(deltaTT, phyllochron,  &
            hasFlagLeafLiguleAppeared, leafNumber, phase)
    !leafNumber: 5.41
    print *, "leafNumber estimated :"
    print *, leafNumber

END PROGRAM
